<a href="https://colab.research.google.com/github/sourabhprk31/sourabhprk31.github.io/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
model_checkpoint = "deepset/roberta-base-squad2"
batch_size = 8

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
!pip install datasets
from datasets import load_dataset, load_metric

In [6]:
datasets = load_dataset("./testsq")

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/10a8ae4af1bd16511a11e728927318d36cc75d8aea10fb45e27fb514e95bed26)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,id,context,question,answers
0,304372,"The Operational Software shall set bloodAir = ENABLED, if all the following conditions are TRUE \nC1: (valve1 = OFF)\nC2: (valve2 = Enabled) \nC3: (valve3 = disabled)",what shall be set?,"{'text': ['bloodAir = ENABLED'], 'answer_start': [35]}"
1,304358,"The Operational Software shall set FD_LLFS_Rev_Snsr = TRUE, if\nC1: ((Upper_Level_Float_Switch = WET) AND \nC2: (Low_Level_Float_Switch_1 = DRY)) OR\nC3: ((Upper_Level_Float_Switch = WET) AND \nC4: (Low_Level_Float_Switch_2 = DRY))",what is C2?,"{'text': ['Low_Level_Float_Switch_1 = DRY'], 'answer_start': [111]}"
2,304402,"The Operational Software shall set temparature = false, \nC1 is (valve1 = ON) OR \nC2 is (valve2 = OFF)",what is C1?,"{'text': ['valve1 = ON'], 'answer_start': [64]}"
3,304357,"The Operational Software shall set FD_LLFS_Rev_Snsr = TRUE, if\nC1: ((Upper_Level_Float_Switch = WET) AND \nC2: (Low_Level_Float_Switch_1 = DRY)) OR\nC3: ((Upper_Level_Float_Switch = WET) AND \nC4: (Low_Level_Float_Switch_2 = DRY))",what is C1?,"{'text': ['Upper_Level_Float_Switch = WET'], 'answer_start': [69]}"
4,304386,"The Operational Software shall set command1 = DISABLED, if all the following conditions are TRUE \nC1: (Display1 = Enabled)\nC2: (Display2 = Disabled) \nC3: (Display3= DISABLED)",what is C2?,"{'text': ['Display2 = Disabled'], 'answer_start': [129]}"
5,304369,"The Operational Software shall set command1 = DISABLED, if all the following conditions are TRUE \nC1: (AFCU = Enabled)\nC2: (Pump = Disabled) \nC3: (valve1= DISABLED)",what shall be set?,"{'text': ['command1 = DISABLED'], 'answer_start': [35]}"
6,304401,"The Operational Software shall set command1 = DISABLED, if all the following conditions are TRUE \nC1: (AFCU = Enabled)\nC2: (Pump = Disabled) \nC3: (valve1= DISABLED)",what is C3?,"{'text': ['valve1= DISABLED'], 'answer_start': [148]}"
7,304381,"The Operational Software shall set Pressure_Indicator = TRUE, if\nC1: (( = WET) AND \nC2: (Switch_1= DRY)) OR\nC3: ((Switch_2 = WET) AND \nC4: (Switch_3 = DRY))",what is C1?,"{'text': [' = WET'], 'answer_start': [71]}"
8,304370,"The Operational Software shall set Pressure = false, \nC1 is (Valve10 = ON) OR \nC2 is (valve20 = OFF)",what shall be set?,"{'text': ['Pressure = false'], 'answer_start': [35]}"
9,304359,"The Operational Software shall set FD_LLFS_Rev_Snsr = TRUE, if\nC1: ((Upper_Level_Float_Switch = WET) AND \nC2: (Low_Level_Float_Switch_1 = DRY)) OR\nC3: ((Upper_Level_Float_Switch = WET) AND \nC4: (Low_Level_Float_Switch_2 = DRY))",what is C3?,"{'text': ['Upper_Level_Float_Switch = WET'], 'answer_start': [153]}"


In [9]:
!pip install transformers
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [11]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [12]:
pad_on_right = tokenizer.padding_side == "right"
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [13]:
features = prepare_train_features(datasets['train'][:5])

In [14]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/10a8ae4af1bd16511a11e728927318d36cc75d8aea10fb45e27fb514e95bed26/cache-19718fc2bee9991a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/10a8ae4af1bd16511a11e728927318d36cc75d8aea10fb45e27fb514e95bed26/cache-a3ee1cde93b93f91.arrow


In [15]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [16]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
)

In [17]:
from transformers import default_data_collator

data_collator = default_data_collator

In [18]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [19]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/content/roberta-base-squad2-finetuned-squad is already a clone of https://huggingface.co/prk/roberta-base-squad2-finetuned-squad. Make sure you pull the latest changes with `repo.git_pull()`.


In [20]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 59
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8


Epoch,Training Loss,Validation Loss
1,No log,0.189430


***** Running Evaluation *****
  Num examples = 59
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8, training_loss=0.5826459527015686, metrics={'train_runtime': 319.6552, 'train_samples_per_second': 0.185, 'train_steps_per_second': 0.025, 'total_flos': 11562381485568.0, 'train_loss': 0.5826459527015686, 'epoch': 1.0})

In [21]:
trainer.save_model("test-squad-trained")

Saving model checkpoint to test-squad-trained
Configuration saved in test-squad-trained/config.json
Model weights saved in test-squad-trained/pytorch_model.bin
tokenizer config file saved in test-squad-trained/tokenizer_config.json
Special tokens file saved in test-squad-trained/special_tokens_map.json
Saving model checkpoint to roberta-base-squad2-finetuned-squad
Configuration saved in roberta-base-squad2-finetuned-squad/config.json
Model weights saved in roberta-base-squad2-finetuned-squad/pytorch_model.bin
tokenizer config file saved in roberta-base-squad2-finetuned-squad/tokenizer_config.json
Special tokens file saved in roberta-base-squad2-finetuned-squad/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.37k/473M [00:00<?, ?B/s]

Upload file runs/Mar02_05-42-00_b994f39a7de3/1646199731.9276178/events.out.tfevents.1646199731.b994f39a7de3.10…

Upload file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

Upload file runs/Mar02_05-42-00_b994f39a7de3/events.out.tfevents.1646199731.b994f39a7de3.1064.0:  85%|########…

Upload file runs/Mar02_04-33-18_b994f39a7de3/1646195687.1958776/events.out.tfevents.1646195687.b994f39a7de3.68…

Upload file runs/Mar02_04-33-18_b994f39a7de3/events.out.tfevents.1646195687.b994f39a7de3.68.0: 100%|##########…

To https://huggingface.co/prk/roberta-base-squad2-finetuned-squad
   016f219..74c66d3  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}, 'dataset': {'name': 'squad_v2', 'type': 'squad_v2', 'args': 'squad_v2'}}
To https://huggingface.co/prk/roberta-base-squad2-finetuned-squad
   74c66d3..623bbbe  main -> main



In [22]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [23]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [24]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 59
  Batch size = 8


In [26]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [27]:
max_answer_length = 30

In [39]:
import numpy as np
n_best_size = 2
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 15.629395, 'text': 'Bleed_Air_Valve_Enable_Cmd = DISABLED'},
 {'score': 12.382669, 'text': 'Bleed_Air_Valve_Enable_Cmd = DISABLED,'}]

In [40]:
data = datasets['train']

In [41]:
data

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 59
})

In [42]:
result = trainer.predict(data)

The following columns in the test set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: context, question, answers, id.
***** Running Prediction *****
  Num examples = 0
  Batch size = 8


In [43]:
result

PredictionOutput(predictions=None, label_ids=None, metrics={'test_runtime': 0.0167, 'test_samples_per_second': 0.0, 'test_steps_per_second': 0.0})

In [36]:
validation_features

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 59
})

In [38]:
raw_predictions[0]

(array([[-0.17630492, -8.415636  , -8.989284  , ..., -9.606938  ,
         -9.606938  , -9.606938  ],
        [ 1.0770162 , -9.15717   , -9.308976  , ..., -9.672185  ,
         -9.672185  , -9.672185  ],
        [ 1.6881964 , -9.027873  , -9.261365  , ..., -9.748445  ,
         -9.748445  , -9.748445  ],
        ...,
        [ 2.2392347 , -9.131917  , -9.3323145 , ..., -9.560285  ,
         -9.560285  , -9.560285  ],
        [ 1.922902  , -9.175668  , -9.342538  , ..., -9.578956  ,
         -9.578956  , -9.578956  ],
        [ 1.8584809 , -9.125518  , -9.359092  , ..., -9.768561  ,
         -9.768561  , -9.768561  ]], dtype=float32),
 array([[ 0.07827985, -7.632202  , -7.1390862 , ..., -8.303071  ,
         -8.303071  , -8.303071  ],
        [ 1.6327776 , -8.440453  , -8.251814  , ..., -8.763561  ,
         -8.763561  , -8.763561  ],
        [ 2.4211924 , -8.258228  , -8.241779  , ..., -8.546389  ,
         -8.546389  , -8.546389  ],
        ...,
        [ 2.603921  , -8.4758835 , -8.3

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("prk/roberta-base-squad2-finetuned-squad")

In [5]:
tokenizer = AutoTokenizer.from_pretrained('prk/roberta-base-squad2-finetuned-squad')

In [6]:
from transformers import pipeline
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [7]:
context = "The Operational Software shall set Bleed_Air_Valve_Enable_Cmd = ENABLED, if all the following conditions are TRUE C1: (valve10 = OFF) C2: (valve11 = Enabled) C3: (valve3 = disabled)"
question = "what shall be set?"
nlp({
    'question': question,
    'context': context
})

{'answer': 'Bleed_Air_Valve_Enable_Cmd = ENABLED',
 'end': 71,
 'score': 0.008538927882909775,
 'start': 35}